# Time Unit Digits Plots

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import scipy.stats as stats
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_reporting_times
from carwatch_analysis.datasets import CarWatchDatasetProcessed
from carwatch_analysis.plotting import time_unit_digits_histogram, time_unit_digits_histogram_grid

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs_grid = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 1.0}
sns.set_theme(**theme_kwargs_grid)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
deploy_type = "develop"

In [ ]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
data_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])
data_path

In [ ]:
dataset = CarWatchDatasetProcessed(data_path)
dataset

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
stats_path = result_path.joinpath("statistics")

img_path = result_path.joinpath("plots/graphical_abstract")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img/graphical_abstract")

bp.utils.file_handling.mkdirs([result_path, stats_path, img_path, paper_img_path])

## Load Data

### Cortisol Samples

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_reporting_types.csv")
cort_samples = load_cortisol_samples_reporting_times(cort_path)
# rename condition for
cort_samples = cort_samples.rename({"Spontaneous": "Spontaneous Awakening"}, level="condition")
cort_samples.head()

In [ ]:
conditions = ["Spontaneous Awakening", "Known Alarm", "Unknown Alarm"]
reporting_types_fine = [
    "Naive",
    "Selfreport without App",
    "Selfreport with App",
    "App",
    "Sensor + Selfreport without App",
    "Sensor + Selfreport with App",
    "Sensor + App",
]
reporting_types_coarse = ["Naive", "Selfreport", "App", "Sensor + Selfreport", "Sensor + App"]

In [ ]:
dict_unit_digits = {}

## Awakening and Sampling Time Unit Digits

In [ ]:
reporting_type_order = ["Selfreport", "App"]
condition_order = ["Known Alarm", "Spontaneous Awakening", "Unknown Alarm"]

### Data Preparation

In [ ]:
cort_samples = cort_samples.assign(
    **{
        "sample_minute": cort_samples["sample_time"].dt.components["minutes"] % 10,
        "wake_onset_minute": cort_samples["wake_onset"].dt.components["minutes"] % 10,
    }
)

cort_samples.head()

## Plots

### Awakening Times

In [ ]:
data_analysis = cort_samples.reindex(reporting_type_order, level="reporting_type")
x = "wake_onset_minute"
ylim = [0, 65]

fig, axs = time_unit_digits_histogram_grid(
    data=data_analysis,
    x=x,
    condition_order=condition_order,
    reporting_type_order=reporting_type_order,
    suptitle="Awakening Times",
    ylim=ylim,
    figsize=(8, 9),
)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_unit_digits_awakening_time", path, ["pdf", "png"], dpi=300)

### Sampling Times

In [ ]:
ylim = [0, 35]
data_analysis = cort_samples.reindex(reporting_type_order, level="reporting_type")

fig, axs = time_unit_digits_histogram(
    data=data_analysis,
    x="sample_minute",
    reporting_type_order=reporting_type_order,
    suptitle="Sampling Times",
    ylim=ylim,
    figsize=(8, 3),
)


for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_unit_digits_sampling_time_combined", path, ["pdf", "png"])

## Export

In [ ]:
bp.io.write_pandas_dict_excel(dict_unit_digits, result_path.joinpath("time_unit_digits_results.xlsx"))